In [13]:
import pandas as pd
import duckdb as ddb
import json

In [25]:
def update():
    with open('response.json') as f:
        data = json.load(f)
    location_df = pd.DataFrame([data['location']])
    weather_df = pd.DataFrame([data['current']])
    weather_df
    flattened_data = {**data["location"], **data["current"]}

    # Convert to a DataFrame
    df = pd.DataFrame([flattened_data])
    df = df.drop(columns='condition')
    df.columns.value_counts().sum()
    conn = ddb.connect('../weatherdata')
    conn.execute("CREATE SCHEMA IF NOT EXISTS raw")
    conn.execute("""
             CREATE TABLE IF NOT EXISTS raw.DAILY_WEATHER_DATA (
                    name VARCHAR(100),
                    region VARCHAR(100),
                    country VARCHAR(100),
                    lat FLOAT,
                    lon FLOAT,
                    tz_id VARCHAR(100),
                    localtime_epoch BIGINT,
                    localtime DATETIME,
                    last_updated_epoch BIGINT,
                    last_updated DATETIME,
                    temp_c FLOAT,
                    temp_f FLOAT,
                    is_day TINYINT,
                    wind_mph FLOAT,
                    wind_kph FLOAT,
                    wind_degree SMALLINT,
                    wind_dir VARCHAR(10),
                    pressure_mb FLOAT,
                    pressure_in FLOAT,
                    precip_mm FLOAT,
                    precip_in FLOAT,
                    humidity TINYINT,
                    cloud TINYINT,
                    feelslike_c FLOAT,
                    feelslike_f FLOAT,
                    windchill_c FLOAT,
                    windchill_f FLOAT,
                    heatindex_c FLOAT,
                    heatindex_f FLOAT,
                    dewpoint_c FLOAT,
                    dewpoint_f FLOAT,
                    vis_km FLOAT,
                    vis_miles FLOAT,
                    uv FLOAT,
                    gust_mph FLOAT,
                    gust_kph FLOAT,
                    PRIMARY KEY (localtime_epoch)
                );

             """)
    conn.execute("INSERT INTO raw.DAILY_WEATHER_DATA SELECT * FROM df")
    conn.close()

    

In [ ]:
# with open('response.json') as f:
#     data = json.load(f)

In [ ]:
# location_df = pd.DataFrame([data['location']])
# weather_df = pd.DataFrame([data['current']])
# weather_df

,last_updated_epoch,last_updated,temp_c,temp_f,is_day,condition,wind_mph,wind_kph,wind_degree,wind_dir,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,1733286600,2024-12-04 10:00,24.1,75.4,1,"{'text': 'Partly cloudy', 'icon': '//cdn.weath...",10.3,16.6,107,ESE,...,73.8,25.1,77.2,16.4,61.6,6.0,3.0,6.5,11.8,19.0


In [ ]:
# flattened_data = {**data["location"], **data["current"]}

# # Convert to a DataFrame
# df = pd.DataFrame([flattened_data])
# df

,name,region,country,lat,lon,tz_id,localtime_epoch,localtime,last_updated_epoch,last_updated,...,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,vis_km,vis_miles,uv,gust_mph,gust_kph
0,Bengaluru,Karnataka,India,12.9833,77.5833,Asia/Kolkata,1733286920,2024-12-04 10:05,1733286600,2024-12-04 10:00,...,73.8,25.1,77.2,16.4,61.6,6.0,3.0,6.5,11.8,19.0


In [ ]:
# df.columns

Index(['name', 'region', 'country', 'lat', 'lon', 'tz_id', 'localtime_epoch',
       'localtime', 'last_updated_epoch', 'last_updated', 'temp_c', 'temp_f',
       'is_day', 'condition', 'wind_mph', 'wind_kph', 'wind_degree',
       'wind_dir', 'pressure_mb', 'pressure_in', 'precip_mm', 'precip_in',
       'humidity', 'cloud', 'feelslike_c', 'feelslike_f', 'windchill_c',
       'windchill_f', 'heatindex_c', 'heatindex_f', 'dewpoint_c', 'dewpoint_f',
       'vis_km', 'vis_miles', 'uv', 'gust_mph', 'gust_kph'],
      dtype='object')

In [ ]:
# df = df.drop(columns='condition')

In [ ]:
# df.columns.value_counts().sum()

np.int64(36)

In [ ]:
# conn = ddb.connect('../weatherdata')


In [ ]:
# conn.execute("CREATE SCHEMA IF NOT EXISTS raw")

In [ ]:
# conn.execute("""
#              CREATE TABLE IF NOT EXISTS raw.DAILY_WEATHER_DATA (
#                     name VARCHAR(100),
#                     region VARCHAR(100),
#                     country VARCHAR(100),
#                     lat FLOAT,
#                     lon FLOAT,
#                     tz_id VARCHAR(100),
#                     localtime_epoch BIGINT,
#                     localtime DATETIME,
#                     last_updated_epoch BIGINT,
#                     last_updated DATETIME,
#                     temp_c FLOAT,
#                     temp_f FLOAT,
#                     is_day TINYINT,
#                     wind_mph FLOAT,
#                     wind_kph FLOAT,
#                     wind_degree SMALLINT,
#                     wind_dir VARCHAR(10),
#                     pressure_mb FLOAT,
#                     pressure_in FLOAT,
#                     precip_mm FLOAT,
#                     precip_in FLOAT,
#                     humidity TINYINT,
#                     cloud TINYINT,
#                     feelslike_c FLOAT,
#                     feelslike_f FLOAT,
#                     windchill_c FLOAT,
#                     windchill_f FLOAT,
#                     heatindex_c FLOAT,
#                     heatindex_f FLOAT,
#                     dewpoint_c FLOAT,
#                     dewpoint_f FLOAT,
#                     vis_km FLOAT,
#                     vis_miles FLOAT,
#                     uv FLOAT,
#                     gust_mph FLOAT,
#                     gust_kph FLOAT,
#                     PRIMARY KEY (localtime_epoch)
#                 );

#              """)

In [ ]:
# conn.execute("INSERT INTO raw.DAILY_WEATHER_DATA SELECT * FROM df")

ConstraintException: Constraint Error: Duplicate key "localtime_epoch: 1733286920" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).

In [ ]:
# conn.close()

In [28]:
update()

ConstraintException: Constraint Error: Duplicate key "localtime_epoch: 1733287531" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (https://duckdb.org/docs/sql/indexes).